In [19]:
import pandas as pd

In [20]:
# make the dictionary whose keys are file names and values are 1-dimensional ndarray.
# we don't need teampping here because it doesn't have "Tournament" feature.
matches_data_files = ["draft_phase", "eco_rounds", "eco_stats", "kills_stats",
              "kills", "maps_played", "maps_scores", "overview", "rounds_kills",
              "scores", "win_loss_methods_count", "win_loss_methods_round_number"]
tournaments = {}
for file in matches_data_files:
    filename = f"../data/vct_2022/matches/{file}.csv"
    tournaments[file] = pd.read_csv(filename).Tournament.unique()


In [21]:
print("The number of unique tournaments of each file.")
for file, df in tournaments.items():
    print(f"{file}: {len(df)}")

The number of unique tournaments of each file.
draft_phase: 53
eco_rounds: 58
eco_stats: 58
kills_stats: 58
kills: 58
maps_played: 58
maps_scores: 58
overview: 58
rounds_kills: 58
scores: 58
win_loss_methods_count: 58
win_loss_methods_round_number: 58


In [22]:
files_by_tnmt_len = {}

for file, df in tournaments.items():
    if len(df) in files_by_tnmt_len:
        files_by_tnmt_len[len(df)].append(file)
    else:
        files_by_tnmt_len[len(df)] = [file]

files_by_tnmt_len

{53: ['draft_phase'],
 58: ['eco_rounds',
  'eco_stats',
  'kills_stats',
  'kills',
  'maps_played',
  'maps_scores',
  'overview',
  'rounds_kills',
  'scores',
  'win_loss_methods_count',
  'win_loss_methods_round_number']}

In [23]:
def not_in_the_other(a: list, b: list) -> list: 
    '''
    input: a pair of lists
    return: the list of elements which are in the list a, but not in b.
    '''
    not_in_b = []
    for i in a:
        if i not in b:
            not_in_b.append(i)
            
    return not_in_b

def all_pairs(k: int) -> list:
    '''
    input: a postive integer k bigger than 1
    return: list of all pairs from 0,1,2,...,k-1.  Two elements in a pair are different.
    '''
    if k < 2:
        return print("input needs to be an integer bigger than 1")
    
    else:
        pairs = []
        for i in range(k):
            n = i
            m = i+1
            while m < k:
                pairs.append((n,m))
                m += 1
        return pairs

In [24]:
for length, file_list in files_by_tnmt_len.items():
    print(f"Check if all the files, having {length} many tournaments, share the same tournaments.")
    if len(file_list) == 1:
        print(f"There is only one file with {length} many tournaments.")
    else:
        pair_ind = all_pairs(len(file_list))
        diff_pair_ind = []

        for i, j in pair_ind:
            not_in_intersection = not_in_the_other(tournaments[file_list[i]], tournaments[file_list[j]])
            if not_in_intersection != []:
                print(not_in_intersection)
                diff_pair_ind.append((i, j))

        if len(diff_pair_ind) > 0:
            for i, j in diff_pair_ind:
                print(f"{file_list[i]} and {file_list[j]} have different tournaments.")
        else:
            print("All files have the same tournaments as we want.")
    print("====================")

Check if all the files, having 53 many tournaments, share the same tournaments.
There is only one file with 53 many tournaments.
Check if all the files, having 58 many tournaments, share the same tournaments.
All files have the same tournaments as we want.


In [25]:
### Is tournaments["draft_phase"] a subset of tournaments["eco_rounds"]?
sample_file_names = ["draft_phase", "eco_rounds"]
sample_dict = {}
pairs_in_smpl_dict = all_pairs(len(sample_file_names))

for i, j in pairs_in_smpl_dict:
    missing_tnmt = not_in_the_other(tournaments[sample_file_names[i]], tournaments[sample_file_names[j]])
    if missing_tnmt:
        print(missing_tnmt, f"is in {sample_file_names[i]}, but not in {sample_file_names[j]}")
    else:
        print(f"All the tournaments in {sample_file_names[i]} are also in {sample_file_names[j]}")

All the tournaments in draft_phase are also in eco_rounds


##### I would like to check if missing tournaments are important ones:

In [26]:
not_in_the_other(tournaments["eco_rounds"], tournaments["draft_phase"])

['Road to VCT 2022: Stage 2',
 'Champions Tour Latin America South Stage 1: Challengers',
 'Road to VCT 2022',
 'Champions Tour Turkey Stage 1: Challengers 1',
 'Champions Tour CIS Stage 1: Challengers 1']

In [27]:
### List of tournaments I want to include in my data set.
# Stage 1
stage_1 = ["Champions Tour Stage 1: EMEA Challengers",
"Champions Tour North America Stage 1: Challengers",
"Champions Tour Latin America Stage 1: Playoffs",
"Champions Tour LATAM/BR Stage 1: Last Chance Qualifier",
"Champions Tour Asia-Pacific Stage 1: Challengers Playoffs",
"Champions Tour Korea Stage 1: Challengers",
"Champions Tour Japan Stage 1: Challengers Playoffs",
"Valorant Champions Tour Stage 1: Masters Reykjavík"]

# Stage 2
stage_2 = ["Champions Tour EMEA Stage 2: Challengers",
"Champions Tour North America Stage 2: Challengers",
"Champions Tour LATAM/BR Stage 2: Last Chance Qualifier",
"Champions Tour Brazil Stage 2: Challengers",
"Champions Tour Asia-Pacific Stage 2: Challengers Playoffs",
"Champions Tour Korea Stage 2: Challengers",
"Champions Tour Japan Stage 2: Challengers Playoffs",
"Valorant Champions Tour Stage 2: Masters Copenhagen"]

# Champions
champions = ["Champions Tour EMEA: Last Chance Qualifier",
"Champions Tour North America: Last Chance Qualifier",
"Champions Tour South America: Last Chance Qualifier",
"Champions Tour Asia-Pacific: Last Chance Qualifier",
"Champions Tour East Asia: Last Chance Qualifier",
"Valorant Champions 2022"]

my_tournaments_2022 = {"stage 1": stage_1, "stage 2": stage_2, "champions": champions}

vct_2022 = ["stage 1", "stage 2", "champions"]

for stage in vct_2022:
    for tnmt in my_tournaments_2022[stage]:
        if tnmt not in tournaments["draft_phase"]:
            print(f"{tnmt} is not in draft_phase.")
        
        if tnmt not in tournaments["eco_rounds"]:
            print(f"{tnmt} is not in eco_rounds.")

By runnning the cell above, we can confirm that all tournaments in my_tournaments_2022 are in our dataset.

I am gonna use "overview" and "maps_scores" data to construct new dataframe with team composition infomation.

In [28]:
overview = pd.read_csv("../data/vct_2022/matches/overview.csv")

In [29]:
overview.head()

,Tournament,Stage,Match Type,Match Name,Map,Player,Team,Agents,Rating,Average Combat Score,...,Deaths,Assists,Kills - Deaths (KD),"Kill, Assist, Trade, Survive %",Average Damage per Round,Headshot %,First Kills,First Deaths,Kills - Deaths (FKD),Side
0,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,Benkai,Paper Rex,fade,1.31,242.0,...,16.0,13.0,6.0,88%,156.0,34%,1.0,0.0,1.0,both
1,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,Benkai,Paper Rex,fade,1.18,243.0,...,9.0,5.0,2.0,75%,167.0,32%,0.0,0.0,0.0,attack
2,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,Benkai,Paper Rex,fade,1.45,242.0,...,7.0,8.0,4.0,100%,146.0,35%,1.0,0.0,1.0,defend
3,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,d4v41,Paper Rex,sage,1.15,219.0,...,16.0,7.0,4.0,88%,140.0,33%,0.0,4.0,-4.0,both
4,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,d4v41,Paper Rex,sage,1.04,222.0,...,9.0,4.0,1.0,100%,147.0,29%,0.0,2.0,-2.0,attack


In [30]:
ind1 = overview.Tournament.isin(champions)
ind2 = (overview.Map != "All Maps")
ind3 = (overview.Side == "both")

ind = (ind1 & ind2 & ind3)
vct_champions_2022_agent_sel_with_no_result = overview[ind][["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team", "Agents"]]
vct_champions_2022_agent_sel_with_no_result

,Tournament,Stage,Match Type,Match Name,Map,Team,Agents
0,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,Paper Rex,fade
3,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,Paper Rex,sage
6,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,Paper Rex,astra
9,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,Paper Rex,chamber
12,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,Paper Rex,raze
...,...,...,...,...,...,...,...
11307,Champions Tour Asia-Pacific: Last Chance Quali...,Main Event,Grand Final,BOOM Esports vs ONIC G,Icebox,ONIC G,viper
11310,Champions Tour Asia-Pacific: Last Chance Quali...,Main Event,Grand Final,BOOM Esports vs ONIC G,Icebox,ONIC G,sage
11313,Champions Tour Asia-Pacific: Last Chance Quali...,Main Event,Grand Final,BOOM Esports vs ONIC G,Icebox,ONIC G,jett
11316,Champions Tour Asia-Pacific: Last Chance Quali...,Main Event,Grand Final,BOOM Esports vs ONIC G,Icebox,ONIC G,sova


In [31]:
vct_champions_2022_comps_with_no_result = vct_champions_2022_agent_sel_with_no_result.groupby(
                                    ["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team"], as_index=False).agg(lambda x: ",".join(sorted(list(x))))

vct_champions_2022_comps_with_no_result

,Tournament,Stage,Match Type,Match Name,Map,Team,Agents
0,Champions Tour Asia-Pacific: Last Chance Quali...,Main Event,Grand Final,BOOM Esports vs ONIC G,Bind,BOOM Esports,"astra,fade,raze,skye,viper"
1,Champions Tour Asia-Pacific: Last Chance Quali...,Main Event,Grand Final,BOOM Esports vs ONIC G,Bind,ONIC G,"astra,chamber,raze,sova,viper"
2,Champions Tour Asia-Pacific: Last Chance Quali...,Main Event,Grand Final,BOOM Esports vs ONIC G,Fracture,BOOM Esports,"breach,brimstone,chamber,kayo,neon"
3,Champions Tour Asia-Pacific: Last Chance Quali...,Main Event,Grand Final,BOOM Esports vs ONIC G,Fracture,ONIC G,"breach,brimstone,chamber,kayo,neon"
4,Champions Tour Asia-Pacific: Last Chance Quali...,Main Event,Grand Final,BOOM Esports vs ONIC G,Haven,BOOM Esports,"astra,breach,jett,killjoy,sova"
...,...,...,...,...,...,...,...
535,Valorant Champions 2022,Playoffs,Upper Semifinals,OpTic Gaming vs XSET,Ascent,XSET,"jett,kayo,killjoy,omen,sova"
536,Valorant Champions 2022,Playoffs,Upper Semifinals,OpTic Gaming vs XSET,Haven,OpTic Gaming,"chamber,fade,killjoy,omen,phoenix"
537,Valorant Champions 2022,Playoffs,Upper Semifinals,OpTic Gaming vs XSET,Haven,XSET,"breach,chamber,fade,omen,raze"
538,Valorant Champions 2022,Playoffs,Upper Semifinals,OpTic Gaming vs XSET,Pearl,OpTic Gaming,"astra,chamber,fade,kayo,viper"


In [32]:
maps_scores = pd.read_csv("../data/vct_2022/matches/maps_scores.csv")
maps_scores
cham_ind_maps_scores = maps_scores.Tournament.isin(champions)
vct_champions_2022_maps_scores = maps_scores[cham_ind_maps_scores][["Tournament", "Stage", "Match Type", "Match Name", "Map",
                                                                "Team A", "Team A Score", "Team B", "Team B Score"]]
vct_champions_2022_maps_scores

,Tournament,Stage,Match Type,Match Name,Map,Team A,Team A Score,Team B,Team B Score
0,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,Paper Rex,13,EDward Gaming,11
1,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Icebox,Paper Rex,5,EDward Gaming,13
2,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Haven,Paper Rex,13,EDward Gaming,8
3,Valorant Champions 2022,Group Stage,Opening (A),Leviatán vs Team Liquid,Haven,Leviatán,13,Team Liquid,10
4,Valorant Champions 2022,Group Stage,Opening (A),Leviatán vs Team Liquid,Ascent,Leviatán,13,Team Liquid,10
...,...,...,...,...,...,...,...,...,...
265,Champions Tour Asia-Pacific: Last Chance Quali...,Main Event,Lower Final,BLEED vs ONIC G,Haven,BLEED,12,ONIC G,14
266,Champions Tour Asia-Pacific: Last Chance Quali...,Main Event,Grand Final,BOOM Esports vs ONIC G,Bind,BOOM Esports,13,ONIC G,10
267,Champions Tour Asia-Pacific: Last Chance Quali...,Main Event,Grand Final,BOOM Esports vs ONIC G,Haven,BOOM Esports,14,ONIC G,12
268,Champions Tour Asia-Pacific: Last Chance Quali...,Main Event,Grand Final,BOOM Esports vs ONIC G,Fracture,BOOM Esports,11,ONIC G,13


In [33]:
vct_champions_2022_team_comp = pd.merge(vct_champions_2022_comps_with_no_result, vct_champions_2022_maps_scores)


In [37]:
vct_champions_2022_team_comp["A win"] = (vct_champions_2022_team_comp["Team A Score"] > vct_champions_2022_team_comp["Team B Score"])
vct_champions_2022_team_comp["B win"] = (vct_champions_2022_team_comp["Team B Score"] > vct_champions_2022_team_comp["Team A Score"])

print("The total number of games:", len(vct_champions_2022_team_comp)//2)
print("The number of unique comp:", len(vct_champions_2022_team_comp.Agents.unique()))
vct_champions_2022_team_comp.head(20)

The total number of games: 270
The number of unique comp: 127


,Tournament,Stage,Match Type,Match Name,Map,Team,Agents,Team A,Team A Score,Team B,Team B Score,A win,B win
0,Champions Tour Asia-Pacific: Last Chance Quali...,Main Event,Grand Final,BOOM Esports vs ONIC G,Bind,BOOM Esports,"astra,fade,raze,skye,viper",BOOM Esports,13,ONIC G,10,True,False
1,Champions Tour Asia-Pacific: Last Chance Quali...,Main Event,Grand Final,BOOM Esports vs ONIC G,Bind,ONIC G,"astra,chamber,raze,sova,viper",BOOM Esports,13,ONIC G,10,True,False
2,Champions Tour Asia-Pacific: Last Chance Quali...,Main Event,Grand Final,BOOM Esports vs ONIC G,Fracture,BOOM Esports,"breach,brimstone,chamber,kayo,neon",BOOM Esports,11,ONIC G,13,False,True
3,Champions Tour Asia-Pacific: Last Chance Quali...,Main Event,Grand Final,BOOM Esports vs ONIC G,Fracture,ONIC G,"breach,brimstone,chamber,kayo,neon",BOOM Esports,11,ONIC G,13,False,True
4,Champions Tour Asia-Pacific: Last Chance Quali...,Main Event,Grand Final,BOOM Esports vs ONIC G,Haven,BOOM Esports,"astra,breach,jett,killjoy,sova",BOOM Esports,14,ONIC G,12,True,False
5,Champions Tour Asia-Pacific: Last Chance Quali...,Main Event,Grand Final,BOOM Esports vs ONIC G,Haven,ONIC G,"chamber,kayo,neon,omen,sova",BOOM Esports,14,ONIC G,12,True,False
6,Champions Tour Asia-Pacific: Last Chance Quali...,Main Event,Grand Final,BOOM Esports vs ONIC G,Icebox,BOOM Esports,"chamber,killjoy,sage,sova,viper",BOOM Esports,13,ONIC G,11,True,False
7,Champions Tour Asia-Pacific: Last Chance Quali...,Main Event,Grand Final,BOOM Esports vs ONIC G,Icebox,ONIC G,"chamber,jett,sage,sova,viper",BOOM Esports,13,ONIC G,11,True,False
8,Champions Tour Asia-Pacific: Last Chance Quali...,Main Event,Lower Final,BLEED vs ONIC G,Ascent,BLEED,"fade,jett,kayo,omen,sage",BLEED,13,ONIC G,10,True,False
9,Champions Tour Asia-Pacific: Last Chance Quali...,Main Event,Lower Final,BLEED vs ONIC G,Ascent,ONIC G,"astra,chamber,fade,jett,kayo",BLEED,13,ONIC G,10,True,False


There are too many compositions compared to the number of games.  I think it's better to have adopt the way of general team composition. \newline
For example, if we have "jett,kayo,killjoy,omen,sova", then this comp is 1 duelist, 2 initiators, 1 controller, 1 sentinel.  \newline

I am gonna make new columns, called duelist, initiator, controller, sentinel, and then make comp column whose values are "DICS" where each letter donotes the number of each role.

In [38]:
def get_comp(agents: list) -> str:
    """ 
    input: string of five agents that order of agents are sorted aphabetically and seperated by ","
    return: string "dics" where d: the # of duelist(s)
                                i: the # of initiator(s)
                                c: the # of controller(s)
                                s: the # of sentinel(s)
    """
    roles = {"duelist": {"jett", "phoenix", "reyna", "raze", "yoru", "neon"},
             "initiator": {"sova", "breach", "skye", "kayo", "fade", "gekko"},
             "controller": {"brimstone", "omen", "viper", "astra", "harbor"},
             "sentinel": {"cypher", "sage", "killjoy", "chamber"}}
    
    agentlist = agents.split(",")
    dul = 0
    ini = 0
    con = 0
    sen = 0

    for agent in agentlist:
        if agent in roles["duelist"]:
            dul += 1
        elif agent in roles["initiator"]:
            ini += 1
        elif agent in roles["controller"]:
            con += 1
        else:
            sen += 1

    assert (dul + ini + con + sen) == 5
    
    composition = str(dul) + str(ini) + str(con) + str(sen)

    return composition

In [39]:
### Let's test the get_comp funtion!!

get_comp("breach,chamber,neon,omen,skye")

'1211'

In [40]:
vct_champions_2022_team_comp["comp code"] = vct_champions_2022_team_comp["Agents"].apply(get_comp)
triple_sentinel_ind = vct_champions_2022_team_comp["comp code"] == "0113"

vct_champions_2022_team_comp

,Tournament,Stage,Match Type,Match Name,Map,Team,Agents,Team A,Team A Score,Team B,Team B Score,A win,B win,comp code
0,Champions Tour Asia-Pacific: Last Chance Quali...,Main Event,Grand Final,BOOM Esports vs ONIC G,Bind,BOOM Esports,"astra,fade,raze,skye,viper",BOOM Esports,13,ONIC G,10,True,False,1220
1,Champions Tour Asia-Pacific: Last Chance Quali...,Main Event,Grand Final,BOOM Esports vs ONIC G,Bind,ONIC G,"astra,chamber,raze,sova,viper",BOOM Esports,13,ONIC G,10,True,False,1121
2,Champions Tour Asia-Pacific: Last Chance Quali...,Main Event,Grand Final,BOOM Esports vs ONIC G,Fracture,BOOM Esports,"breach,brimstone,chamber,kayo,neon",BOOM Esports,11,ONIC G,13,False,True,1211
3,Champions Tour Asia-Pacific: Last Chance Quali...,Main Event,Grand Final,BOOM Esports vs ONIC G,Fracture,ONIC G,"breach,brimstone,chamber,kayo,neon",BOOM Esports,11,ONIC G,13,False,True,1211
4,Champions Tour Asia-Pacific: Last Chance Quali...,Main Event,Grand Final,BOOM Esports vs ONIC G,Haven,BOOM Esports,"astra,breach,jett,killjoy,sova",BOOM Esports,14,ONIC G,12,True,False,1211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,Valorant Champions 2022,Playoffs,Upper Semifinals,OpTic Gaming vs XSET,Ascent,XSET,"jett,kayo,killjoy,omen,sova",OpTic Gaming,11,XSET,13,False,True,1211
536,Valorant Champions 2022,Playoffs,Upper Semifinals,OpTic Gaming vs XSET,Haven,OpTic Gaming,"chamber,fade,killjoy,omen,phoenix",OpTic Gaming,13,XSET,9,True,False,1112
537,Valorant Champions 2022,Playoffs,Upper Semifinals,OpTic Gaming vs XSET,Haven,XSET,"breach,chamber,fade,omen,raze",OpTic Gaming,13,XSET,9,True,False,1211
538,Valorant Champions 2022,Playoffs,Upper Semifinals,OpTic Gaming vs XSET,Pearl,OpTic Gaming,"astra,chamber,fade,kayo,viper",OpTic Gaming,13,XSET,8,True,False,0221


- "Agents" and "comp code" are for "Team A"

In [41]:
vct_champions_2022 = vct_champions_2022_team_comp[["Tournament", "Stage", "Match Type", "Team", "Map", "Agents", "comp code", "A win", "B win"]]

In [48]:
vct_champions_2022_win_rate_by_comp_code = vct_champions_2022[["Map", "comp code", "A win"]].groupby(["Map", "comp code"], as_index=False).agg("mean")
vct_champions_2022_win_rate_by_comp_code = vct_champions_2022_win_rate_by_comp_code\
                            .set_index(["Map", "comp code"])\
                            .rename(columns={"A win": "win pct by map-comp"})
vct_champions_2022_win_rate_by_comp_code.sort_values(["Map", "win pct by map-comp"], ascending=False)

win pct by map-comp
Map      comp code                     
Split    2012                  1.000000
         1121                  0.777778
         1220                  0.500000
         1211                  0.400000
         1112                  0.250000
         0221                  0.000000
Pearl    0212                  1.000000
         0221                  1.000000
         1112                  1.000000
         1211                  0.666667
         1121                  0.200000
Icebox   1121                  1.000000
         1112                  0.631579
         0113                  0.600000
         1211                  0.600000
         0212                  0.545455
         0221                  0.500000
Haven    0311                  1.000000
         1112                  0.700000
         1211                  0.525641
         2111                  0.500000
         0212                  0.000000
         1121                  0.000000
Fracture 1220                  1.000000
         1121                  0.727273
         1112                  0.600000
         1211                  0.523810
         2111                  0.500000
Breeze   0221                  1.000000
         1220                  1.000000
         0311                  0.750000
         1211                  0.471429
Bind     2111                  1.000000
         2210                  0.600000
         1220                  0.575758
         1121                  0.533333
         1211                  0.333333
Ascent   0221                  1.000000
         1112                  1.000000
         2210                  1.000000
         1310                  0.750000
         0311                  0.555556
         1211                  0.552239
         0212                  0.363636

In [53]:
df = vct_champions_2022\
    .set_index(["Map", "comp code"])\
    .join(vct_champions_2022_win_rate_by_comp_code)\
    .reset_index()\
    .sort_values("win pct by map-comp", ascending=False)

In [54]:
df.sort_values(["Map", "Team", "win pct by map-comp"], ascending=False)

,Map,comp code,Tournament,Stage,Match Type,Team,Agents,A win,B win,win pct by map-comp
193,Split,1211,Champions Tour East Asia: Last Chance Qualifier,Main Event,Lower Round 2,REJECT,"astra,chamber,fade,kayo,raze",False,True,0.400000
217,Split,1211,Champions Tour East Asia: Last Chance Qualifier,Main Event,Upper Quarterfinals,REJECT,"astra,chamber,fade,kayo,raze",False,True,0.400000
145,Split,1121,Champions Tour EMEA: Last Chance Qualifier,Main Event,Upper Quarterfinals,Natus Vincere,"chamber,fade,omen,raze,viper",True,False,0.777778
177,Split,1121,Champions Tour East Asia: Last Chance Qualifier,Main Event,Lower Final,NORTHEPTION,"chamber,fade,omen,raze,viper",True,False,0.777778
227,Split,1121,Champions Tour East Asia: Last Chance Qualifier,Main Event,Upper Semifinals,NORTHEPTION,"chamber,fade,omen,raze,viper",False,True,0.777778
...,...,...,...,...,...,...,...,...,...,...
382,Ascent,0212,Valorant Champions 2022,Group Stage,Decider (D),100 Thieves,"chamber,fade,kayo,omen,sage",False,True,0.363636
238,Ascent,0212,Champions Tour North America: Last Chance Qual...,Main Event,Lower Final,100 Thieves,"chamber,fade,kayo,omen,sage",False,True,0.363636
262,Ascent,0212,Champions Tour North America: Last Chance Qual...,Main Event,Lower Round 3,100 Thieves,"chamber,fade,kayo,omen,sage",True,False,0.363636
276,Ascent,0212,Champions Tour North America: Last Chance Qual...,Main Event,Upper Quarterfinals,100 Thieves,"chamber,fade,kayo,omen,sage",False,True,0.363636


In [58]:
import pandas as pd
from IPython.display import HTML

def side_by_side(*dfs):
    html = '<div style="display:flex">'
    for df in dfs:
        html += '<div style="margin-right: 2em">'
        html += df.to_html()
        html += '</div>'
    html += '</div>'
    display(HTML(html))

In [59]:
wincount_by_comp_code = df[["Map", "comp code", "A win"]].groupby(["Map", "comp code"], as_index=False).agg("sum")
wincount_by_comp_code = wincount_by_comp_code\
                                .set_index(["Map", "comp code"])\
                                .rename(columns={"A win": "win count"})

losscount_by_comp_code = df[["Map", "comp code", "B win"]].groupby(["Map", "comp code"], as_index=False).agg("sum")
losscount_by_comp_code = losscount_by_comp_code\
                                .set_index(["Map", "comp code"])\
                                .rename(columns={"B win": "loss count"})
side_by_side(wincount_by_comp_code.sort_values(["Map", "win count"], ascending=False),
losscount_by_comp_code.sort_values(["Map", "loss count"], ascending=False))

In [61]:
def get_history(codelist: list, maplist: list):
    """ 
    
    """
    index = (df["comp code"].isin(codelist)) & (df["Map"].isin(maplist))
    display(df[index])

In [63]:
display(get_history(["0221", "1211", "1112"], ["Pearl"]))
display(get_history(["1121", "1211"], ["Pearl"]))

,Map,comp code,Tournament,Stage,Match Type,Team,Agents,A win,B win,win pct by map-comp
455,Pearl,0221,Valorant Champions 2022,Group Stage,Winner's (B),OpTic Gaming,"astra,chamber,fade,kayo,viper",True,False,1.000000
454,Pearl,0221,Valorant Champions 2022,Group Stage,Winner's (B),LOUD,"astra,chamber,fade,kayo,viper",True,False,1.000000
417,Pearl,1112,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex,"astra,chamber,fade,raze,sage",True,False,1.000000
416,Pearl,0221,Valorant Champions 2022,Group Stage,Opening (A),EDward Gaming,"astra,chamber,fade,kayo,viper",True,False,1.000000
372,Pearl,1112,Valorant Champions 2022,Group Stage,Decider (A),Paper Rex,"astra,chamber,fade,raze,sage",True,False,1.000000
538,Pearl,0221,Valorant Champions 2022,Playoffs,Upper Semifinals,OpTic Gaming,"astra,chamber,fade,kayo,viper",True,False,1.000000
539,Pearl,1211,Valorant Champions 2022,Playoffs,Upper Semifinals,XSET,"astra,chamber,fade,kayo,neon",True,False,0.666667
503,Pearl,1211,Valorant Champions 2022,Playoffs,Lower Round 2,XSET,"astra,chamber,fade,kayo,neon",True,False,0.666667
529,Pearl,1211,Valorant Champions 2022,Playoffs,Upper Quarterfinals,XSET,"astra,chamber,fade,kayo,neon",True,False,0.666667
445,Pearl,1211,Valorant Champions 2022,Group Stage,Opening (D),FNATIC,"astra,chamber,fade,neon,skye",False,True,0.666667


None

,Map,comp code,Tournament,Stage,Match Type,Team,Agents,A win,B win,win pct by map-comp
539,Pearl,1211,Valorant Champions 2022,Playoffs,Upper Semifinals,XSET,"astra,chamber,fade,kayo,neon",True,False,0.666667
503,Pearl,1211,Valorant Champions 2022,Playoffs,Lower Round 2,XSET,"astra,chamber,fade,kayo,neon",True,False,0.666667
529,Pearl,1211,Valorant Champions 2022,Playoffs,Upper Quarterfinals,XSET,"astra,chamber,fade,kayo,neon",True,False,0.666667
445,Pearl,1211,Valorant Champions 2022,Group Stage,Opening (D),FNATIC,"astra,chamber,fade,neon,skye",False,True,0.666667
528,Pearl,1211,Valorant Champions 2022,Playoffs,Upper Quarterfinals,FNATIC,"astra,chamber,fade,neon,skye",True,False,0.666667
461,Pearl,1211,Valorant Champions 2022,Group Stage,Winner's (C),XSET,"astra,chamber,fade,kayo,neon",False,True,0.666667
395,Pearl,1121,Valorant Champions 2022,Group Stage,Elimination (B),ZETA DIVISION,"chamber,omen,raze,skye,viper",False,True,0.200000
444,Pearl,1121,Valorant Champions 2022,Group Stage,Opening (D),100 Thieves,"astra,fade,raze,sage,viper",False,True,0.200000
373,Pearl,1121,Valorant Champions 2022,Group Stage,Decider (A),Team Liquid,"astra,chamber,fade,phoenix,viper",True,False,0.200000
460,Pearl,1121,Valorant Champions 2022,Group Stage,Winner's (C),FunPlus Phoenix,"cypher,fade,omen,raze,viper",False,True,0.200000


None

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
ax = sns.scatterplot(data=df, x="Map", y="A win pct by comp", hue="comp code")
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
{"Ascent", "Haven", "Breeze", "Icebox", "Split", "Pearl", "Bind", "Fracture"}

sns.countplot(data=vct_champions_2022_team_comp[vct_champions_2022_team_comp["Map"]=="Ascent"],
             x="comp code")

In [ ]:
ignore_tournament_name

In [ ]:
wincount_by_comp_code = ignore_tournament_name[["Map", "comp code", "A win"]].groupby(["Map", "comp code"], as_index=False).agg("sum")

wincount_by_comp_code

wincount_by_comp_code = wincount_by_comp_code\
                                .set_index(["Map", "comp code"])\
                                .rename(columns={"A win": "win count"})
display(wincount_by_comp_code.sort_values(["Map", "win count"], ascending=False))

losscount_by_comp_code = ignore_tournament_name[["Map", "comp code", "B win"]].groupby(["Map", "comp code"], as_index=False).agg("sum")

losscount_by_comp_code

losscount_by_comp_code = losscount_by_comp_code\
                                .set_index(["Map", "comp code"])\
                                .rename(columns={"B win": "loss count"})
display(losscount_by_comp_code.sort_values(["Map", "loss count"], ascending=True))

In [ ]:
display(ignore_tournament_name)